In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [7]:
train = pd.read_csv('.\\data\\FIFA_train.csv')
test = pd.read_csv('.\\data\\FIFA_test.csv')
submission = pd.read_csv('.\\data\\submission.csv')

In [8]:
train.head()

id          name  age      continent contract_until position prefer_foot  \
0   0      L. Messi   31  south america           2021       ST        left   
1   3        De Gea   27         europe           2020       GK       right   
2   7     L. Suárez   31  south america           2021       ST       right   
3   8  Sergio Ramos   32         europe           2020       DF       right   
4   9      J. Oblak   25         europe           2021       GK       right   

   reputation  stat_overall  stat_potential  stat_skill_moves        value  
0         5.0            94              94               4.0  110500000.0  
1         4.0            91              93               1.0   72000000.0  
2         5.0            91              91               3.0   80000000.0  
3         4.0            91              91               3.0   51000000.0  
4         3.0            90              93               1.0   68000000.0

In [9]:
test.head()

id               name  age      continent contract_until position  \
0   1  Cristiano Ronaldo   33         europe           2022       ST   
1   2          Neymar Jr   26  south america           2022       ST   
2   4       K. De Bruyne   27         europe           2023       MF   
3   5          E. Hazard   27         europe           2020       ST   
4   6          L. Modrić   32         europe           2020       MF   

  prefer_foot  reputation  stat_overall  stat_potential  stat_skill_moves  
0       right         5.0            94              94               5.0  
1       right         5.0            92              93               5.0  
2       right         4.0            91              92               4.0  
3       right         4.0            91              91               4.0  
4       right         4.0            91              91               4.0

In [10]:
submission.head()

id  value
0   1      0
1   2      0
2   4      0
3   5      0
4   6      0

In [11]:
def con_period(x):
    if x == 'Dec 31, 2018' :
        return '2019'
    elif x == 'Jun 30, 2020' :
        return '2020.5'
    elif x == 'Jun 30, 2019' :
        return '2019.5'
    elif x == 'May 31, 2020' :
        return '2020.3333'
    elif x == 'May 31, 2019' :
        return '2019.3333'
    elif x == 'Jan 31, 2019' :
        return '2019.0833'
    elif x == 'Jan 1, 2019' :
        return '2019'
    elif x == 'Jan 12, 2019' :
        return '2019.034'
    elif x == 'Dec 31, 2019' :
        return '2020'
    elif x == 'Jun 1, 2019' :
        return '2019.416'
    else :
        return x

In [12]:
train.contract_until = train.contract_until.apply(con_period).astype('float64') - 2018
test.contract_until = test.contract_until.apply(con_period).astype('float64') - 2018

In [13]:
train[['age', 'stat_potential']] = np.log1p(train[['age', 'stat_potential']])
test[['age', 'stat_potential']] = np.log1p(test[['age', 'stat_potential']])

In [14]:
X = train[['age', 'continent', 'contract_until', 'position', 'reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']]
y = np.log1p(train['value'])

In [15]:
X = pd.get_dummies(columns = ['continent', 'position'], data = X)

In [16]:
target = test[['age', 'continent', 'contract_until', 'position', 'reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']]

In [17]:
target = pd.get_dummies(columns = ['continent', 'position'], data = target)

In [18]:
kf = KFold(n_splits = 10, random_state = 521, shuffle = True)

In [19]:
ngb = NGBRegressor(random_state = 521, verbose = 500, n_estimators = 500)

In [20]:
ngb_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    ngb.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in ngb.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in ngb.predict(target)]) / 10
    ngb_pred += sub_pred
print(f'{ngb.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

[iter 0] loss=1.7562 val_loss=0.0000 scale=1.0000 norm=1.2861
[iter 100] loss=0.7611 val_loss=0.0000 scale=2.0000 norm=1.1141
[iter 200] loss=-0.1028 val_loss=0.0000 scale=2.0000 norm=0.9194
[iter 300] loss=-0.7609 val_loss=0.0000 scale=2.0000 norm=0.8144
[iter 400] loss=-1.0723 val_loss=0.0000 scale=2.0000 norm=0.8506
[iter 0] loss=-1.1726 val_loss=0.0000 scale=1.0000 norm=0.4586
[iter 100] loss=-1.2324 val_loss=0.0000 scale=1.0000 norm=0.4658
[iter 200] loss=-1.2790 val_loss=0.0000 scale=2.0000 norm=0.9459
[iter 300] loss=-1.3129 val_loss=0.0000 scale=1.0000 norm=0.4786
[iter 400] loss=-1.3410 val_loss=0.0000 scale=2.0000 norm=0.9640
[iter 0] loss=-1.3254 val_loss=0.0000 scale=2.0000 norm=1.0137
[iter 100] loss=-1.3735 val_loss=0.0000 scale=1.0000 norm=0.4876
[iter 200] loss=-1.3927 val_loss=0.0000 scale=2.0000 norm=0.9750
[iter 300] loss=-1.4115 val_loss=0.0000 scale=1.0000 norm=0.4871
[iter 400] loss=-1.4346 val_loss=0.0000 scale=1.0000 norm=0.4853
[iter 0] loss=-1.4245 val_loss=0.

In [42]:
model_ngb = sm.OLS(X, y)
result = model_ngb.fit()

result.summary()

ValueError: shapes (8932,15) and (8932,15) not aligned: 15 (dim 1) != 8932 (dim 0)

In [21]:
rf = RandomForestRegressor(random_state = 521, n_estimators = 150)

In [22]:
rf_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in rf.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in rf.predict(target)]) / 10
    rf_pred += sub_pred
print(f'{rf.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

RandomForestRegressor의 10fold 평균 RMSE는 807354.2566586821


In [23]:
etc = ExtraTreesRegressor(random_state = 521, n_estimators = 500)

In [24]:
etc_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    etc.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in etc.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in etc.predict(target)]) / 10
    etc_pred += sub_pred
print(f'{etc.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

ExtraTreesRegressor의 10fold 평균 RMSE는 748777.7449780542


In [25]:
gb = GradientBoostingRegressor(random_state = 521, max_depth = 5)

In [26]:
gb_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    gb.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in gb.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in gb.predict(target)]) / 10
    gb_pred += sub_pred
print(f'{gb.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

GradientBoostingRegressor의 10fold 평균 RMSE는 590051.6390746763


In [27]:
cb = CatBoostRegressor(random_state = 521, silent = True, depth = 3)

In [28]:
cb_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    cb.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in cb.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in cb.predict(target)]) / 10
    cb_pred += sub_pred
print(f'{cb.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

CatBoostRegressor의 10fold 평균 RMSE는 444907.98380159435


In [29]:
lgbm = LGBMRegressor(random_state = 521, max_depth = 4, n_estimators = 1000)

In [30]:
lgbm_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(target)]) / 10
    lgbm_pred += sub_pred
print(f'{lgbm.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

LGBMRegressor의 10fold 평균 RMSE는 600903.6346210752


In [31]:
xgb = XGBRegressor(random_state = 521, max_depth = 5)

In [32]:
xgb_pred = np.zeros((target.shape[0]))
rmse_list = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb.fit(tr_x, tr_y)
    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(val_x)])

    rmse = np.sqrt(mean_squared_error(np.expm1(val_y), pred))
    rmse_list.append(rmse)
    
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(target)]) / 10
    xgb_pred += sub_pred
print(f'{xgb.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}')

XGBRegressor의 10fold 평균 RMSE는 581307.7178958185


In [34]:
submission['value'] = (gb_pred + rf_pred + etc_pred + lgbm_pred + cb_pred + ngb_pred + xgb_pred) / 7


array([5.95594447e+07, 9.37229710e+07, 7.91129374e+07, ...,
       5.95277318e+04, 4.97725431e+04, 5.04208175e+04])

In [35]:
q1 = submission['value'].quantile(0.004)
q2 = submission['value'].quantile(0.99)

submission['value'] = submission['value'].apply(lambda x: x if x > q1 else x * 0.9)
submission['value'] = submission['value'].apply(lambda x: x if x < q2 else x * 1.1)

In [38]:
submission['value'] = submission['value'].astype('int')
submission

id      value
0         1   65515389
1         2  103095268
2         4   87024231
3         5   91661198
4         6   70043135
...     ...        ...
3823  16924      58399
3824  16929      49855
3825  16932      59527
3826  16937      49772
3827  16943      50420

[3828 rows x 2 columns]